In [3]:
import numpy as np
from sklearn.decomposition import PCA 
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler

In [4]:
#缩小图片规格，以减少计算量及因图片错误引起的无关因素规格为113x90
def depart(initial_matrix):
    kl = initial_matrix[15:]
    tk = []
    for i in range(113):    
        t = kl[i][20:110]
        tk.append(t)
    return tk
#离散处理数据像素，使其可能取值数量为17
def lisan(ol):
    for i in range(113):
        for j in range(90):
            ol[i][j] = (ol[i][j]//15)*15+7
    return ol
def cou_male(a):
    '''
    统计男性数量
    '''
    t = 0
    for i in a:
        if i==1:
            t += 1
    return t
#寻找男性坐标
def findex(a):
    num = []
    for i in range(len(a)):
        if a[i] == 1:
            num.append(i)
    return num
#计算权重，注意，此时特征向量已经拉长处理
def weigh(data_x,data_y):
    kp = findex(data_y)#记录男性坐标
    k = [i for i in range(len(data_y))]
    #得出女性坐标
    for i in kp:
        k.remove(i)
    al = [0]*len(data_x[0])#建立特征向量的空数组
    #循环求解
    for n in kp:
        for t in k:
            for i in range(len(data_x[0])):
                #不同则加一
                if data_x[n][i].all()!=data_x[t][i].all():
                    al[i] += 1
    he = sum(al)
    tp = []
    for i in range(len(al)):
        tp.append(al[i]/he)
    return tp

In [5]:
# 源数据处理
all_file = os.listdir('./face/rawdata')
# 读取所有data_x数据,  有问题的数据：第1186个，2412号；第1190个，2416号 ；2021号
data_x = np.zeros((len(all_file),128,128))
data_y = np.zeros(len(all_file))
data1_x = np.zeros((len(all_file),113,90))
# 将两个数据标签整理再一起
f = open('./face/faceDR','r')
r1 = f.readlines()
f.close()

f = open('./face/faceDS','r')
r2 = f.readlines()
f.close()
for i in r2:
    r1.append(i)

# 整理出data_x,和data_y
all_file = os.listdir('./face/rawdata')
for i in range(len(all_file)):
    with open('./face/rawdata/'+all_file[i],'rb') as f:
        x = np.fromfile(f,dtype=np.ubyte)
        t = x.reshape(128,-1)
        data_x[i] = t
        for j in r1:
            if all_file[i] in j:
                if 'female' in j:
                    data_y[i] = 0
                elif 'male' in j:
                    data_y[i] = 1
                else:
                    data_y[i] = None
                break

                
#缩小图片，及离散处理
for i in range(len(all_file)):
    data1_x[i] = depart(data_x[i])
#    data1_x[i] = lisan(data1_x[i])
#进一步处理
face_data = data1_x.reshape(data1_x.shape[0],-1)#拉长数组
face_target = data_y
#PCA降维
pca = PCA(n_components = 200).fit(face_data)
face_data = pca.transform(face_data)


X_train,X_test,Y_train,Y_test = train_test_split(face_data,face_target,shuffle = True,random_state=60,test_size = 0.3)

In [6]:
#归一化
mms = MinMaxScaler().fit(X_train)
X_train_ = mms.transform(X_train)
X_test_=mms.transform(X_test)

In [7]:
#gnb = GaussianNB().fit(X_train,Y_train) 
#mnb = MultinomialNB().fit(X_train,Y_train)
#bnb_ = BernoulliNB(binarize=0.0).fit(X_train_,Y_train) 
bnb = BernoulliNB().fit(X_train,Y_train) 
#cnb = ComplementNB().fit(X_train,Y_train) 

In [8]:
#acc_score = gnb.score(X_test,Y_test)#高斯分布
#acc_score1 = mnb.score(X_test,Y_test)#多项式朴素贝叶斯分类器MultinomialNB
acc_score2 = bnb.score(X_test,Y_test)#伯劳尼朴素贝叶斯分类器BernoulliNB
#acc_score21 = bnb_.score(X_test_,Y_test)
#acc_score3 = cnb.score(X_test,Y_test)
#print(acc_score,acc_score1,acc_score2,acc_score3)
print(acc_score2)

0.7372384937238494


In [7]:
Y_pred = mnb.predict(X_test)
from sklearn.metrics import confusion_matrix as CM
CM(Y_test,Y_pred)

array([[274, 191],
       [173, 557]], dtype=int64)

In [9]:
#cv300 = cross_val_score(bnb,face_data,face_target,cv=5)
#cv400 = cross_val_score(bnb,face_data,face_target,cv=5)
#cv500 = cross_val_score(bnb,face_data,face_target,cv=5)
cv200 = cross_val_score(bnb,face_data,face_target,cv=5)
#cv100 = cross_val_score(bnb,face_data,face_target,cv=5)
#print(sum(cv300)/5)
#print(sum(cv100)/5)
print(sum(cv200)/5)
#print(sum(cv400)/5)
#print(sum(cv500)/5)
print(cv200)

0.6685910733088277
[0.61104141 0.69636136 0.70012547 0.68090452 0.65452261]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#各个模型效果对比图

fig = plt.figure()
x = [1,2,3,4,5]
yerr = np.linspace(0.05, 0.2, 5)
plt.errorbar(x, cv100,  yerr=yerr, uplims=True, lolims=True,
             label='PCA_n=100,average:65.35%')
plt.errorbar(x, cv200,  yerr=yerr, uplims=True, lolims=True,
             label='PCA_n=200,average:65.82%')
plt.errorbar(x, cv300,  yerr=yerr, uplims=True, lolims=True,
             label='PCA_n=300,average:64.67%')
plt.errorbar(x, cv400,  yerr=yerr, uplims=True, lolims=True,
             label='PCA_n=400,average:63.37%')
plt.errorbar(x, cv500,  yerr=yerr, uplims=True, lolims=True,
             label='PCA_n=500,average:63.04%')
plt.grid()
plt.xlabel('Cross Validation Time')
plt.ylabel('Accuracy')
plt.legend(loc='lower left')
plt.show()

In [10]:
clf = AdaBoostClassifier(base_estimator=bnb, n_estimators=150)
scores = cross_val_score(clf, face_data,face_target, cv=5)
print('Adaboost-NB AUC：',scores.mean())

Adaboost-NB AUC： 0.6879201528344357
